# CUDA编程模型--- 执行流 和 运行库

#### 1.CUDA流
CUDA程序的并行层次主要有两个，一个是核函数内部的并行，一个是核函数的外部的并行。我们之前讨论的都是核函数的内部的并行。核函数外部的并行主要指：
- 核函数计算与数据传输之间的并行
- 主机计算与数据传输之间的并行
- 不同的数据传输之间的并行
- 核函数计算与主机计算之间的并行
- 不同核函数之间的并行


CUDA流表示一个GPU操作队列，该队列中的操作将以添加到流中的先后顺序而依次执行。我们的所有CUDA操作都是在流中进行的，虽然我们可能没发现，但是有我们前面的例子中的指令，内核启动，都是在CUDA流中进行的，只是这种操作是隐式的，所以肯定还有显式的，所以，流分为：
- 隐式声明的流，我们叫做空流
- 显式声明的流，我们叫做非空流

基于流的异步内核启动和数据传输支持以下类型的粗粒度并发：
- 重叠主机和设备计算
- 重叠主机计算和主机设备数据传输
- 重叠主机设备数据传输和设备计算
- 并发设备计算（多个设备）

接下来，我们就完成下面这个核函数，在两个流并发的实现：
```C++
__global__ void kernel( int *a, int *b, int *c ) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        int idx1 = (idx + 1) % 256;
        int idx2 = (idx + 2) % 256;
        float   as = (a[idx] + a[idx1] + a[idx2]) / 3.0f;
        float   bs = (b[idx] + b[idx1] + b[idx2]) / 3.0f;
        c[idx] = (as + bs) / 2;
    }
}
```

创建[stream.cu](stream.cu)文件，详情请参考[result1.cu](result1-stream.cu)

修改Makefile，利用Makefile编译，并执行

In [3]:
!make -f Makefile_stream

/usr/local/cuda/bin/nvcc  -arch=compute_80 -code=sm_80 stream.cu -o ./stream


In [4]:
!./stream

Time taken:  17.8 ms


利用nvprof测试程序性能

In [5]:
# !sudo /usr/local/cuda/bin/nvprof ./stream

!/usr/local/cuda/bin/nsys nvprof --print-api-trace  ./stream

The --print-api-trace  switch is ignored by nsys.

Time taken:  16.0 ms
Generating '/tmp/nsys-report-18d8.qdstrm'
[1/7] [========================100%] report2.nsys-rep
[2/7] [========================100%] report2.sqlite
[3/7] Executing 'nvtxsum' stats report
SKIPPED: /mnt/CUDA_on_ARM/04_4.2/report2.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cudaapisum' stats report

CUDA API Statistics:

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)     Med (ns)    Min (ns)  Max (ns)   StdDev (ns)          Name         
 --------  ---------------  ---------  -----------  -----------  --------  ---------  -----------  ---------------------
     56.8        210899907          2  105449953.5  105449953.5      1126  210898781  149127162.0  cudaEventCreate      
     27.1        100507050          3   33502350.0   32961255.0  32765564   34780231    1110994.4  cudaHostAlloc        
     11.1         41349375          3   13783125.0   13563678.0  13306769   14478928     616122

编辑[stream2.cu](stream2.cu) 删除其中一个流，并测试性能，如果遇到麻烦，请参考[result2.cu](result2.cu)

利用Makefile文件编译，并执行程序

In [8]:
!make -f Makefile_stream2

/usr/local/cuda/bin/nvcc  -arch=compute_80 -code=sm_80 stream2.cu -o ./stream2
stream2.cu(34): warning #177-D: variable "stream1" was declared but never referenced



In [9]:
!./stream2

Time taken:  29.7 ms


#### 2.cuBLAS  
cuBLAS 库是基于 NVIDIA®CUDA™ 运行时的 BLAS（基本线性代数子程序）的实现。它允许用户访问 NVIDIA GPU 的计算资源。


在[cublas_gemm.cu](cublas_gemm.cu)中使用```cublasDgemm()```函数，如果遇到麻烦，请参考[result3.cu](result3.cu)

修改Makefile文件，这里需要将```$(CC)  $(TEST_SOURCE) -o  $(TARGETBIN)``` 修改为```$(CC)  $(TEST_SOURCE) -lcublas -o  $(TARGETBIN)```  

编译，并执行程序

In [14]:
!make -f Makefile_cublas

/usr/local/cuda/bin/nvcc  -arch=compute_80 -code=sm_80  cublas_gemm.cu -lcublas -o  ./cublas_gemm


In [15]:
!./cublas_gemm

A = 
  0.000000  2.000000  4.000000
  1.000000  3.000000  5.000000
B = 
  0.000000  3.000000
  1.000000  4.000000
  2.000000  5.000000
C = A x B = 
 10.000000 28.000000
 13.000000 40.000000


利用nvprof查看程序性能

In [16]:
# !sudo /usr/local/cuda/bin/nvprof ./cublas_gemm
!/usr/local/cuda/bin/nsys nvprof --print-api-trace  ./cublas_gemm

The --print-api-trace  switch is ignored by nsys.

A = 
  0.000000  2.000000  4.000000
  1.000000  3.000000  5.000000
B = 
  0.000000  3.000000
  1.000000  4.000000
  2.000000  5.000000
C = A x B = 
 10.000000 28.000000
 13.000000 40.000000
Generating '/tmp/nsys-report-56e0.qdstrm'
[1/7] [========================100%] report3.nsys-rep
[2/7] [========================100%] report3.sqlite
[3/7] Executing 'nvtxsum' stats report
SKIPPED: /mnt/CUDA_on_ARM/04_4.2/report3.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cudaapisum' stats report

CUDA API Statistics:

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)    Med (ns)  Min (ns)   Max (ns)   StdDev (ns)            Name          
 --------  ---------------  ---------  -----------  --------  --------  ----------  -----------  ------------------------
     69.6       1074339890          8  134292486.3   15847.5      1552  1073851749  379639284.2  cudaFree                
     16.3        251425458          4   6285

课后作业：
1. 尝试调用cublas做矩阵乘法和向量加法操作，跟之前自己写的程序对比，查看性能差距，并分析可能改进的地方？
2. 如果本地文件存储着2个1000000*1000000的矩阵，我们想将这两个矩阵进行乘积，如何操作？